# Simple EDA with Python

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Import Data
df_as = pd.read_csv("./Data/AllstarFull.csv") #All Stars
df_bat = pd.read_csv("./Data/Batting.csv") #batting (reg season)
df_postbat = pd.read_csv("./Data/BattingPost.csv") #batting (postseason)
df_field = pd.read_csv("./Data/Fielding.csv") #fielding
df_pitch = pd.read_csv("./Data/Pitching.csv") #pitching
df_postpitch = pd.read_csv("./Data/PitchingPost.csv") #Postseason Pitching
df_sal = pd.read_csv("./Data/Salaries.csv") #salaries
df_people = pd.read_csv("./Data/People.csv") #player names

In [3]:
df_as.head()

,playerID,yearID,gameNum,gameID,teamID,lgID,GP,startingPos
0,gomezle01,1933.0,0.0,ALS193307060,NYA,AL,1,1.0
1,ferreri01,1933.0,0.0,ALS193307060,BOS,AL,1,2.0
2,gehrilo01,1933.0,0.0,ALS193307060,NYA,AL,1,3.0
3,gehrich01,1933.0,0.0,ALS193307060,DET,AL,1,4.0
4,dykesji01,1933.0,0.0,ALS193307060,CHA,AL,1,5.0


In [ ]:
#Look at columns of interest